In [ ]:
len(lines1)

In [1]:
import os 
from cv2 import imread, resize
import cv2
import json
import numpy as np

import imgaug.augmenters as iaa


file_path1 = 'C:/Users/user/Desktop/sh/전자제조/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/train.1-1.txt' # 폴더 경로
with open(file_path1) as f:
    lines1 = f.readlines()
    
file_path2 = 'C:/Users/user/Desktop/sh/전자제조/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/validation.1-1.txt' # 폴더 경로
with open(file_path2) as f:
    lines2 = f.readlines()    

save_path = 'C:/Users/user/Desktop/Cognex/'

lines = lines1+lines2
image_names = [line.rstrip('\n') for line in lines]


path = 'C:/Users/user/Desktop/sh/전자제조/d-sub-15pin_for-seoultech/image' # 폴더 경로
os.chdir(path) # 해당 폴더로 이동
files = os.listdir(path) # 해당 폴더에 있는 파일 이름을 리스트 형태로 받음

with open('C:/Users/user/Desktop/sh/전자제조/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as f:

    data = json.load(f)
    
aug_name = "custom_repeat"
class_name = " "

txt_list = []

for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        

        for env_num in range(5):
            env_num = 0
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_repeat/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_repeat/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_repeat/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_repeat/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_repeat/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_repeat/glue/"  +    image_name + ' ' +  str(temp_label))
            
            aug_image =  image
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [4]:
import os 
from cv2 import imread, resize
import cv2
import json
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa

aug_name = "custom_clahe"
class_name = " "


class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size
    
    def __call__(self, img):
        # Convert PIL image to numpy array
        img = np.array(img)
        
        # Convert to grayscale if necessary
        if len(img.shape) == 3 and img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # CLAHE (어두운 부분 강조)
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        cl1 = clahe.apply(img)
        
        if len(cl1.shape) == 2:
            cl1 = np.repeat(cl1[:, :, np.newaxis], 3, axis=2)
            #print(cl1.shape)
            #cl1 = np.transpose(cl1, (1, 2, 0))
            
        # Convert back to PIL Image
        #cl1 = Image.fromarray(cl1)
        return cl1
    
    

for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        

        for env_num in range(5):
            
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            strong_list=[0.5,1,2,3,4]
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_clahe/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_clahe/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_clahe/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_clahe/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_clahe/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_clahe/glue/"  +    image_name + ' ' +  str(temp_label))
            
            transform = CLAHETransform(clip_limit=strong_list[env_num] , tile_grid_size=(16, 16))
            
            aug_image = transform(image)
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [ ]:
import os 
from cv2 import imread, resize
import cv2
import json
import numpy as np

import imgaug.augmenters as iaa

aug_name = "custom_cameraz"
class_name = " "

txt_list = []

for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        

        for env_num in range(5):
            
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_cameraz/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_cameraz/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_cameraz/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_cameraz/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_cameraz/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_cameraz/glue/"  +    image_name + ' ' +  str(temp_label))
            

            
            seq = iaa.Sequential([
                iaa.BlendAlpha((0.25),
                iaa.Affine(rotate=((env_num + 1) * -1, (env_num+ 1) * 1)))
            ])
            
            aug_image =  seq(images=image)
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

        
with open('C:/Users/user/Desktop/Cognex/imagelist/custom_cameraz.txt' , 'w', encoding='UTF-8' ) as f:
    for name in txt_list:
        f.write(name+'\n')

In [ ]:
aug_name = "custom_blur"
txt_list = []

def custom_blur(image, image_number):
    if image_number == '0':
        seq = iaa.Sequential([
            iaa.GaussianBlur(sigma=(0.1))
        ])

    elif image_number == '1':
        seq = iaa.Sequential([
            iaa.GaussianBlur(sigma=(0.2))
        ])

    elif image_number == '2':
        seq = iaa.Sequential([
            iaa.GaussianBlur(sigma=(0.4))
        ])


    elif image_number == '3':
        seq = iaa.Sequential([
            iaa.GaussianBlur(sigma=(0.8))
        ])
        
    elif image_number == '4':
            seq = iaa.Sequential([
            iaa.GaussianBlur(sigma=(1))
        ])
        
    aug_image = seq(images=image)

    return aug_image

for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        
        
        for env_num in range(5):    
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_blur/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_blur/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_blur/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_blur/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_blur/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_blur/glue/"  +    image_name + ' ' +  str(temp_label))
            
            
            aug_image = custom_blur(image, temp2[2])
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [ ]:
temp = file.replace(".","_")
temp

In [2]:
aug_name = "custom_lightness"
txt_list = []

class_name = " "

#homomorphic filter Normalize
def custom_lightness(image, custom_sigma):
    img = image
    img_YUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)    
    y = img_YUV[:,:,0]    
    
    rows = y.shape[0]    
    cols = y.shape[1]
    
    ### illumination elements와 reflectance elements를 분리하기 위해 log를 취함
    imgLog = np.log1p(np.array(y, dtype='float') / 255) # y값을 0~1사이로 조정한 뒤 log(x+1)
    
    ### frequency를 이미지로 나타내면 4분면에 대칭적으로 나타나므로 
    ### 4분면 중 하나에 이미지를 대응시키기 위해 row와 column을 2배씩 늘려줌
    M = 2*rows + 1
    N = 2*cols + 1
    
    ### gaussian mask 생성 sigma = 10
    sigma = custom_sigma
    (X, Y) = np.meshgrid(np.linspace(0, N-1, N), np.linspace(0, M-1, M)) # 0~N-1(and M-1) 까지 1단위로 space를 만듬
    Xc = np.ceil(N/2) # 올림 연산
    Yc = np.ceil(M/2)
    gaussianNumerator = (X - Xc)**2 + (Y - Yc)**2 # 가우시안 분자 생성
    
    ### low pass filter와 high pass filter 생성
    LPF = np.exp(-gaussianNumerator / (2*sigma*sigma))
    HPF = 1 - LPF
    
    ### LPF랑 HPF를 0이 가운데로 오도록iFFT함. 
    ### 사실 이 부분이 잘 이해가 안 가는데 plt로 이미지를 띄워보니 shuffling을 수행한 효과가 났음
    ### 에너지를 각 귀퉁이로 모아 줌
    LPF_shift = np.fft.ifftshift(LPF.copy())
    HPF_shift = np.fft.ifftshift(HPF.copy())
    
    ### Log를 씌운 이미지를 FFT해서 LPF와 HPF를 곱해 LF성분과 HF성분을 나눔
    img_FFT = np.fft.fft2(imgLog.copy(), (M, N))
    img_LF = np.real(np.fft.ifft2(img_FFT.copy() * LPF_shift, (M, N))) # low frequency 성분
    img_HF = np.real(np.fft.ifft2(img_FFT.copy() * HPF_shift, (M, N))) # high frequency 성분
    
    ### 각 LF, HF 성분에 scaling factor를 곱해주어 조명값과 반사값을 조절함
    gamma1 = 0.3
    gamma2 = 1.5
    img_adjusting = gamma1*img_LF[0:rows, 0:cols] + gamma2*img_HF[0:rows, 0:cols]
    
    ### 조정된 데이터를 이제 exp 연산을 통해 이미지로 만들어줌
    img_exp = np.expm1(img_adjusting) # exp(x) + 1
    img_exp = (img_exp - np.min(img_exp)) / (np.max(img_exp) - np.min(img_exp)) # 0~1사이로 정규화
    img_out = np.array(255*img_exp, dtype = 'uint8') # 255를 곱해서 intensity값을 만들어줌
    
    ### 마지막으로 YUV에서 Y space를 filtering된 이미지로 교체해주고 RGB space로 converting
    img_YUV[:,:,0] = img_out
    result = cv2.cvtColor(img_YUV, cv2.COLOR_YUV2BGR)
    
    return result

for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        temp_label = (data['single_image'][file]['class'][0])

        
        for env_num in range(5):    

            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            custom_sigma_list = [4,2,0.1,-2,-4]
            
            aug_image = custom_lightness(image, custom_sigma_list[env_num])
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_lightness/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_lightness/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_lightness/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_lightness/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_lightness/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_lightness/glue/"  +    image_name + ' ' +  str(temp_label))
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [ ]:
from IPython.display import Image
#Image("C:/Users/user/Desktop/sh/전자제조/lightness_sample.png")

In [ ]:
import imageio

In [ ]:
aug_name = "custom_brightness"
txt_list = []
#homomorphic filter Normalize
def custom_brightness(image, custom_hue):
    
    seq = iaa.Sequential([
    iaa.MultiplyHue(custom_hue),
    iaa.pillike.FilterDetail(),
    iaa.pillike.EnhanceBrightness()     
])
    aug_image = seq(images=image)

    return aug_image


for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        temp_label = (data['single_image'][file]['class'][0])
        
        for env_num in range(5):    
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_brightness/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_brightness/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_brightness/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_brightness/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_brightness/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_brightness/glue/"  +    image_name + ' ' +  str(temp_label))
            
            custom_hue_list = [0.05, 0.1, 0.25, 0.5, 1]
            
            aug_image = custom_brightness([image], custom_hue_list[env_num])
            aug_image2 = np.squeeze(np.array(aug_image))
            
            #cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image2)
        
            imageio.imwrite("C:/Users/user/Desktop/Cognex/" +  aug_name + "/" + class_name + image_name , aug_image2)
            
with open('C:/Users/user/Desktop/Cognex/imagelist/custom_brightness.txt' , 'w', encoding='UTF-8' ) as f:
    for name in txt_list:
        f.write(name+'\n')            


In [ ]:
import os 
from cv2 import imread, resize
import cv2
import json
import numpy as np

import imgaug.augmenters as iaa
import imgaug.augmenters.flip as flip

aug_name = "custom_flip"
class_name = " "

txt_list = []


for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        

        for env_num in range(5):
            
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_flip/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_flip/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_flip/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_flip/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_flip/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_flip/glue/"  +    image_name + ' ' +  str(temp_label))

            prob = np.random.rand()
            
            if prob < 0.25 :
                aug_image = image
            elif prob >= 0.25 and prob < 0.5:
                aug_image = flip.fliplr(image)    # 좌우
            elif prob >=0.5 and prob <0.75:
                aug_image = flip.flipud(image)   # 상하
            else:
                temp_image = flip.fliplr(image)
                aug_image = flip.flipud(image) 
            
            
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [ ]:
import os 
from cv2 import imread, resize
import cv2
import json
import numpy as np

import imgaug.augmenters as iaa
import imgaug.augmenters.flip as flip


aug_name = "custom_noise"
class_name = " "

txt_list = []


def custom_noise(image, image_number):
    
    seq = iaa.Sequential([
    iaa.AdditiveGaussianNoise(scale=((int(image_number)+ 1) *0.001*0.5*255))  
])
    aug_image = seq(images=image)

    return aug_image




for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        

        for env_num in range(5):
            
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_noise/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_noise/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_noise/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_noise/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_noise/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_noise/glue/"  +    image_name + ' ' +  str(temp_label))

            aug_image = custom_noise(image, temp2[2])
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [ ]:
import os 
from cv2 import imread, resize
import cv2
import json
import numpy as np

import imgaug.augmenters as iaa
import imgaug.augmenters.flip as flip


aug_name = "custom_dropout"
class_name = " "

txt_list = []


def custom_dropout(image, image_number):
    
    seq = iaa.Sequential([
    iaa.CoarseDropout(0.0005 * (int(image_number)+1) , size_percent=(0.3, 0.5), per_channel=0.3)
])
    aug_image = seq(images=image)

    return aug_image



for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        

        for env_num in range(5):
            
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_dropout/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_dropout/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_dropout/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_dropout/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_dropout/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_dropout/glue/"  +    image_name + ' ' +  str(temp_label))

            aug_image = custom_dropout(image, temp2[2])
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [ ]:
import os 
from cv2 import imread, resize
import cv2
import json
import numpy as np

import imgaug.augmenters as iaa
import imgaug.augmenters.flip as flip

aug_name = "custom_contrast"
class_name = " "

txt_list = []

#영상에서 밝은 부분과 어두운 부분의 차이를 의미
def custom_contrast(image, image_number):
    
    seq = iaa.Sequential([
    iaa.GammaContrast(  1.1**(int(image_number)-2) )
])
    aug_image = seq(images=image)

    return aug_image



for file in files:
    if file in image_names:
        f = imread(file)
        image = resize(f, (512,288))
        #image = normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        temp = file.replace(".","_")
        temp2 = temp.split("_")
        
        temp_label = (data['single_image'][file]['class'][0])
        

        for env_num in range(5):
            
            image_name =  temp2[0] +"_" + aug_name + "_"+ str(env_num) + ".png"
            
            
            if temp_label == 0:
                class_name = 'ok/'
                txt_list.append( "custom_contrast/ok/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 1:
                class_name = 'scratch/'
                txt_list.append( "custom_contrast/scratch/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 2:
                class_name = 'fm/'
                txt_list.append( "custom_contrast/fm/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 3:
                class_name = 'pin/'
                txt_list.append( "custom_contrast/pin/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 4:
                class_name = 'dent/'
                txt_list.append( "custom_contrast/dent/"  +    image_name + ' ' +  str(temp_label))
            elif temp_label == 5:
                class_name = 'glue/'
                txt_list.append( "custom_contrast/glue/"  +    image_name + ' ' +  str(temp_label))

            aug_image = custom_contrast(image, temp2[2])
            
            cv2.imwrite(save_path +  aug_name + "/"  + class_name + image_name , aug_image)

In [ ]:
f = imread(files[0])
image = resize(f, (512,288))

import matplotlib.pyplot as plt

plt.imshow(image)
plt.show()

In [ ]:
import imgaug.augmenters.flip as flip

plt.imshow(flip.fliplr(image))

In [ ]:
plt.imshow(flip.flipud(image))

In [ ]:
iaa.Flipud(0.5)